In [31]:
import datajoint as dj
import numpy as np
import time
from tqdm import tqdm

In [32]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

pinky = dj.create_virtual_module('pinky', 'microns_pinky')
schema = dj.schema('microns_pinky')


In [33]:
@schema
class LeftoverCoarseLabelFinal(dj.Computed):
    definition = """
    # Ensure that Vertex labels for pinky.LeftoverProofreadLabel match the triangle labels, so these are regenerated from the correct triangle labels.
    -> pinky.LeftoverProofreadLabel
    ---
    vertices  : longblob # Corrected vertex labels
    triangles : longblob # Same triangle labels as pinky.LeftoverProofreadLabel
    """
    
    key_source = pinky.LeftoverProofreadLabel & 'status="complete"'
    
    def make(self, key):
        start = time.time()
        
        print(key["segment_id"])
        labels = (pinky.LeftoverProofreadLabel & key).fetch1()
        corrected_vertex_labels = np.zeros(labels['vertices'].shape, np.uint8)
        
        mesh = (pinky.ExcitatoryLeftoverMeshes & 'decimation_ratio=0.35' & dict(segment_id=key['segment_id'])).fetch1()
        mesh_triangles = mesh['triangles']
        
        vertex_label_dict = dict()
        for i, label in enumerate(labels['triangles']):
            triangle = mesh_triangles[i]
            for node in triangle:
                if node in vertex_label_dict:
                    if vertex_label_dict[node] < label:
                        vertex_label_dict[node] = label
                else:
                    vertex_label_dict[node] = label
                
        for node, label in vertex_label_dict.items():
            corrected_vertex_labels[node] = label
            
        self.insert1(dict(key,
                          vertices=corrected_vertex_labels,
                          triangles=labels['triangles']))
        
        print("Segment {} vertex labels regenerated in: {} seconds.".format(key['segment_id'], time.time() - start))

In [34]:
start = time.time()
LeftoverCoarseLabelFinal.populate(reserve_jobs=True)
print(time.time() - start)

648518346341371119
Segment 648518346341371119 vertex labels regenerated in: 0.3344876766204834 seconds.
648518346349386137
Segment 648518346349386137 vertex labels regenerated in: 0.22446084022521973 seconds.
648518346349470171
Segment 648518346349470171 vertex labels regenerated in: 0.344498872756958 seconds.
648518346349471156
Segment 648518346349471156 vertex labels regenerated in: 0.25168418884277344 seconds.
648518346349471500
Segment 648518346349471500 vertex labels regenerated in: 0.2208549976348877 seconds.
648518346349471562
Segment 648518346349471562 vertex labels regenerated in: 0.23991823196411133 seconds.
648518346349471565
Segment 648518346349471565 vertex labels regenerated in: 0.2313392162322998 seconds.
648518346349471910
Segment 648518346349471910 vertex labels regenerated in: 0.23808073997497559 seconds.
648518346349472574
Segment 648518346349472574 vertex labels regenerated in: 0.21092700958251953 seconds.
648518346349472601
Segment 648518346349472601 vertex labels 

In [ ]:
#(schema.jobs & "table_name='__coarse_label_final'").delete()